In [1]:
!pip install pypdf langchain sentence-transformers faiss-cpu


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install -U langchain langchain-text-splitters


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## importacion de celdas

In [1]:
import os
import re
import numpy as np
import faiss

from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama

C:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = Ollama(
    model="llama3",
    temperature=0.2
)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5012\3846646355.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [3]:
print(llm.invoke("Responde en una frase: ¿Qué es una estrella?"))

Una estrella es un gran cuerpo celeste que se forma a partir de la condensación de gas y polvo interestelares, calentado por su propia gravedad hasta alcanzar temperaturas tan altas que producen reacciones nucleares en su interior.


In [4]:
def cargar_pdfs(ruta="../data"):
    textos = []
    print("Buscando PDFs en:", os.path.abspath(ruta))

    for archivo in os.listdir(ruta):
        if archivo.lower().endswith(".pdf"):
            reader = PdfReader(os.path.join(ruta, archivo))
            for pagina in reader.pages:
                texto = pagina.extract_text()
                if texto and len(texto) > 50:
                    textos.append(texto)

    return textos

In [5]:
textos = cargar_pdfs()
print("Páginas cargadas:", len(textos))

Buscando PDFs en: C:\Users\Usuario\Documents\chatbot_universo\data
Páginas cargadas: 1257


In [6]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=80
)

chunks = splitter.split_text("\n".join(textos))
print("Chunks creados:", len(chunks))

Chunks creados: 6359


In [7]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embedder.encode(chunks, show_progress_bar=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

print("FAISS:", index.ntotal)
print("Chunks:", len(chunks))

Batches: 100%|███████████████████████████████████████████████████████████████████████| 199/199 [02:10<00:00,  1.52it/s]

FAISS: 6359
Chunks: 6359


In [8]:
def buscar_contexto(pregunta, k=3):
    pregunta_vec = embedder.encode([pregunta])
    distancias, indices = index.search(pregunta_vec, k)

    contexto = []
    for i in indices[0]:
        if i < len(chunks):
            texto = chunks[i].strip()
            if len(texto) > 80:
                contexto.append(texto[:400])

    return "\n\n".join(contexto)

In [9]:
def preguntar(pregunta):
    contexto = buscar_contexto(pregunta)

    prompt = f"""
Eres un profesor de astronomía.
Responde SOLO con la información del contexto.
Máximo 5 líneas.
Si no hay información suficiente, dilo explícitamente.

Contexto:
{contexto}

Pregunta:
{pregunta}

Respuesta clara y directa:
"""

    return llm.invoke(prompt).strip()

In [12]:
print(preguntar("¿Qué es un planeta?"))

Un planeta es un cuerpo celeste que orbita alrededor de una estrella, como nuestro Sol, y tiene suficiente masa para ser redondo bajo su propia gravedad.


In [13]:
preguntas = [
    "¿Qué es una estrella?",
    "¿Qué es un planeta?",
    "¿Qué es una galaxia?",
    "¿Qué es la Vía Láctea?",
    "¿Qué es un agujero negro?"
]

for p in preguntas:
    print("P:", p)
    print("R:", preguntar(p))
    print("-" * 50)

P: ¿Qué es una estrella?
R: Una estrella es un gran cuerpo celeste que se origina a partir de la gravedad y el calor en una nube de gas y polvo. Es una estrella estándar, evolucionada a través del ciclo de formación y muerte estelar.
--------------------------------------------------
P: ¿Qué es un planeta?
R: En astronomía, un planeta es un cuerpo celeste que orbita alrededor de una estrella y tiene suficiente masa para ser esférico en forma.
--------------------------------------------------
P: ¿Qué es una galaxia?
R: Una galaxia es un sistema de estrellas, gases y polvo que se encuentra unida por la gravedad. Es como una gran ciudad celestial compuesta por millones o incluso billones de estrellas, nebulosas y otros objetos celestiales.
--------------------------------------------------
P: ¿Qué es la Vía Láctea?
R: La Vía Láctea es la galaxia espiral en la que se encuentra el Sol, la Tierra, la Luna y todo el sistema planetario en el que estamos.
--------------------------------------